# Inference with the model on openEO

This notebook demonstrates how to use a trained model for inference with onnxruntime in an udf.

In [18]:
# Make the connection and authenticate
import openeo

#Choose preferred backend
conn = openeo.connect("https://openeo.vito.be/openeo/").authenticate_oidc()
#conn = openeo.connect("https://openeo.dataspace.copernicus.eu/").authenticate_oidc()


Authenticated using refresh token.


A workflow for processing Sentinel-2 Level-2A earth observation data using openEO in orden to obtain the desired data, compatible with the classifier


In [26]:
# Import necessary utility function
from utils import  preprocess_sentinel2_data

spatial_extent = {"west": 12.17,
                  "east": 12.18,
                  "south": 51.46,
                  "north": 51.47}

year = 2020
    
# Rearrange cube from (time, x, y bands) to (x, y, time*bands)
s2_cube = preprocess_sentinel2_data(conn, spatial_extent = spatial_extent, year = year)
s2_cube

Load in the model from the vito artifactory through the UDF

In [27]:

# Supply the model as a URL. The model is stored in artifactory
model_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/photovoltaic/random_forest.onnx"
udf = openeo.UDF.from_file(
    "udf_rf_onnx.py", 
    context={
        "model_url": model_url
    }
)

# Reduce the bands dimesnion to a single prediction using the udf
prediction = s2_cube.reduce_bands(reducer=udf)

Include the ONNX dependencies and excecute the batch job. 
The outcome is saved as a netCDF.

In [28]:
# Add the onnx dependencies to the job options. You can reuse this existing dependencies archive
dependencies_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/openeo/onnx_dependencies_1.16.3.zip"
job_options = {
    "udf-dependency-archives": [
        f"{dependencies_url} #onnx_deps"
    ],
}

prediction_job = prediction.execute_batch(
    "./photovoltaic_prediction.nc",
    job_options=job_options,
    title="photovoltaic_prediction"
)


0:00:00 Job 'j-2404044f0c7a417c9962d0014cb41efc': send 'start'


OpenEoApiError: [500] Internal: Server error: Failed to start batch job (YARN submit failure). (ref: r-240404263abf4131b5643ff78cceeac9)